The purpose of this task is to create an external table on the posts data of the stackoverflow.com website.

Create your own database and `use` it. 
Create external table `posts_sample_external` over the sample dataset with posts in `/data/stackexchange1000` directory. 
Create managed table `posts_sample` and populate with the data from the external table. 
`Posts_sample` table should be partitioned by year and by month of post creation. 
Provide output of query which selects lines number per each partition in the format:

```
year <tab> month <table> lines count
```

where year in YYYY format and month in YYYY-MM format. 
The result is the 3th line of the last query output.

Example:
```
2008 2008-07 123
```


In [1]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

DROP DATABASE IF EXISTS stackexchange CASCADE;

CREATE DATABASE stackexchange LOCATION '/data/stackexchange1000';

USE stackexchange;

Overwriting query.hql


In [2]:
%%writefile -a query.hql

DROP TABLE IF EXISTS posts_sample;

CREATE EXTERNAL TABLE posts_sample(
    Id STRING,
    PostTypeId STRING,
    ParentId STRING,
    CreationDate STRING
)
ROW FORMAT
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
"input.regex" = '(?=.*\\bId=\"(\\d*)\")?(?=.*\\bParentId=\"(\\S*)\")?(?=.*\\bPostTypeId=\"(\\S*)\")?(?=.*\\bCreationDate=\"(\\S*)\")?.*$'
)
LOCATION '/data/stackexchange1000/posts'
tblproperties ("skip.header.line.count"="1");

Appending to query.hql


In [3]:
%%writefile -a query.hql

DROP VIEW IF EXISTS posts_sample_view;

CREATE VIEW posts_sample_view(
    Id,
    PostTypeId,
    ParentId,
    YEAR,
    MONTH
)
PARTITIONED ON (YEAR, MONTH)
AS SELECT
    Id,
    PostTypeId,
    ParentId,
    regexp_extract(CreationDate, "[0-9]{4}", 0),
    regexp_extract(CreationDate, "[0-9]{4}-[0-9]{2}", 0)
FROM posts_sample;

Appending to query.hql


In [4]:
%%writefile -a query.hql

SELECT YEAR, MONTH, count FROM (
    SELECT * FROM (
        SELECT 
            YEAR, 
            MONTH, 
            count(*) as count, 
            row_number() over() as rn 
        FROM posts_sample_view 
        GROUP BY YEAR, MONTH 
        ORDER BY rn DESC 
        LIMIT 3
    ) AS t 
    ORDER BY t.rn ASC 
    LIMIT 1
) AS tt;

Appending to query.hql


In [5]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.056 seconds
OK
Time taken: 0.338 seconds
OK
Time taken: 0.027 seconds
OK
Time taken: 0.097 seconds
OK
Time taken: 0.621 seconds
OK
Time taken: 0.025 seconds
OK
Time taken: 1.098 seconds
Query ID = jovyan_20180609064848_82ebe5a2-3874-422f-ac35-9570141a428c
Total jobs = 4
Launching Job 1 out of 4
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1528502025591_0006, Tr